In [52]:
# pip install fitparse

from fitparse import FitFile
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

# Відкрити FIT файл
fitfile = FitFile('17003786880_ACTIVITY.fit')

# Ініціалізуємо список для зберігання даних
records = []

# Перебираємо записи у файлі
for record in fitfile.get_messages('record'):
    record_data = {}
    for record_field in record:
        record_data[record_field.name] = record_field.value
    records.append(record_data)

# Створюємо DataFrame з отриманих записів
df = pd.DataFrame(records)

df['time_only'] = df['timestamp'].dt.time
# Обчислення ковзного середнього зі вікном в 3 рядки (наприклад)
df['heart_rate_rolling_avg'] = df['heart_rate'].rolling(window=5).mean()


start_time = pd.to_datetime('04:08:00').time()
end_time = pd.to_datetime('04:15:00').time()

# Фільтруємо DataFrame за інтервалом часу
df_filtered = df[(df['time_only'] >= start_time) & (df['time_only'] <= end_time)]


dff = df_filtered[['time_only', 'heart_rate', 'heart_rate_rolling_avg']]
dff.head(1)

,time_only,heart_rate,heart_rate_rolling_avg
293,04:08:00,155,154.8


In [53]:
fig = go.Figure()

# Додавання першої лінії для heart_rate
fig.add_trace(go.Scatter(x=dff['time_only'], y=dff['heart_rate'], mode='lines', name='Heart Rate'))

# Додавання другої лінії для heart_rate_rolling_avg
fig.add_trace(go.Scatter(x=dff['time_only'], y=dff['heart_rate_rolling_avg'], mode='lines', name='Heart Rate Rolling Avg'))

# Вимкнення легенди
fig.update_layout(showlegend=False)

# Відображення графіку
fig.show()


In [54]:
fig = px.line(dff, x="time_only", y="heart_rate_rolling_avg")
fig.show()

{'cadence': 0, 'distance': 0.0, 'enhanced_speed': 0.0, 'fractional_cadence': 0.0, 'heart_rate': 75, 'speed': 0.0, 'timestamp': datetime.datetime(2024, 9, 11, 3, 28, 14)}
{'cadence': 61, 'distance': 0.0, 'enhanced_speed': 0.0, 'fractional_cadence': 0.5, 'heart_rate': 75, 'speed': 0.0, 'timestamp': datetime.datetime(2024, 9, 11, 3, 28, 15)}
{'cadence': 61, 'distance': 8.13, 'enhanced_speed': 1.577, 'fractional_cadence': 0.5, 'heart_rate': 78, 'speed': 1.577, 'timestamp': datetime.datetime(2024, 9, 11, 3, 28, 21)}
{'cadence': 74, 'distance': 11.23, 'enhanced_speed': 1.54, 'fractional_cadence': 0.0, 'heart_rate': 81, 'speed': 1.54, 'timestamp': datetime.datetime(2024, 9, 11, 3, 28, 23)}
{'cadence': 75, 'distance': 15.97, 'enhanced_speed': 1.633, 'fractional_cadence': 0.5, 'heart_rate': 85, 'speed': 1.633, 'timestamp': datetime.datetime(2024, 9, 11, 3, 28, 26)}
{'cadence': 78, 'distance': 21.22, 'enhanced_speed': 1.773, 'fractional_cadence': 0.0, 'heart_rate': 88, 'speed': 1.773, 'timestamp

In [56]:
df.columns

Index(['cadence', 'distance', 'enhanced_speed', 'fractional_cadence',
       'heart_rate', 'speed', 'timestamp', 'altitude', 'enhanced_altitude',
       'position_lat', 'position_long', 'time_only', 'heart_rate_rolling_avg'],
      dtype='object')

In [59]:
df.speed


0      0.000
1      0.000
2      1.577
3      1.540
4      1.633
       ...  
697    2.305
698    2.305
699    2.295
700    2.295
701    2.286
Name: speed, Length: 702, dtype: float64